In [2]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

from prototypes.CUHF2 import *
from prototypes.UNOCI import *
from prototypes.UHF import *
from prototypes.GHF import *
from prototypes.RHF import *
from prototypes.spin import *
from prototypes.optimization import *
from prototypes.FCI import *

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})


## Parameters

In [3]:
basis = "STO-3G"
N_H = 3
N_a = 2
N_b = 1

In [4]:
mol = gqcpy.Molecule.HChain(N_H, 1.4, 0)

In [5]:
_, HFpar = UHF(mol, N_a, N_b, basis)

UHF energy:  -1.5095578776455083


In [6]:
spinor_basis = gqcpy.USpinOrbitalBasis_d(mol, basis)
N = int(spinor_basis.numberOfSpinors()/2)

## Testing

In [7]:
S_aa = spinor_basis.quantize(gqcpy.OverlapOperator()).alpha.parameters()
S_bb = spinor_basis.quantize(gqcpy.OverlapOperator()).beta.parameters()

In [8]:
Sp_a =  np.zeros_like(S_aa)
Sp_b = S_aa

In [9]:
Sm_a = S_bb
Sm_b = np.zeros_like(S_aa)

In [10]:
Smp_a = np.zeros_like(S_aa)
Smp_b = S_aa

In [11]:
Sz2_a = 0.25 * S_aa
Sz2_b = 0.25 * S_bb

In [12]:
Sz_a = spinor_basis.quantize(gqcpy.ElectronicSpin_zOperator()).alpha.parameters()
Sz_b = spinor_basis.quantize(gqcpy.ElectronicSpin_zOperator()).beta.parameters()

In [13]:
Sz = gqcpy.ScalarUSQOneElectronOperator_d(gqcpy.ScalarUSQOneElectronOperatorComponent_d(Sz_a), gqcpy.ScalarUSQOneElectronOperatorComponent_d(Sz_b))
S_mp = gqcpy.ScalarUSQOneElectronOperator_d(gqcpy.ScalarUSQOneElectronOperatorComponent_d(Smp_a), gqcpy.ScalarUSQOneElectronOperatorComponent_d(Smp_b))
Sz2 = gqcpy.ScalarUSQOneElectronOperator_d(gqcpy.ScalarUSQOneElectronOperatorComponent_d(Sz2_a), gqcpy.ScalarUSQOneElectronOperatorComponent_d(-Sz2_b))

In [14]:
spin = Sz + S_mp + Sz2

In [15]:
tensor_ab = gqcpy.ScalarMixedUSQTwoElectronOperatorComponent_d(-1*np.einsum('pr,qs->pqrs', S_aa, S_bb))
tensor_ba = gqcpy.ScalarMixedUSQTwoElectronOperatorComponent_d(-1*np.einsum('pr,qs->pqrs', S_bb, S_aa))

tensor_aa = gqcpy.ScalarPureUSQTwoElectronOperatorComponent_d(np.zeros((N, N, N, N)))
tensor_bb = gqcpy.ScalarPureUSQTwoElectronOperatorComponent_d(np.zeros((N, N, N, N)))

tensor = gqcpy.ScalarUSQTwoElectronOperator_d(tensor_aa, tensor_ab, tensor_ba, tensor_bb)

In [16]:
OneDM = HFpar.calculateScalarBasis1DM()
twoDM = HFpar.calculateScalarBasis2DM()

In [17]:
spin.calculateExpectationValue(OneDM)[0]

1.7499999999999996

In [18]:
tensor.calculateExpectationValue(twoDM)[0]

-0.9848877655048695

In [19]:
S2_val = spin.calculateExpectationValue(OneDM)[0] + tensor.calculateExpectationValue(twoDM)[0]
S2_val

0.7651122344951301

In [20]:
Sz_val = Sz.calculateExpectationValue(OneDM)[0]
Sz_val

0.49999999999999967